In [1]:
import numpy as np
from skimage.exposure import rescale_intensity
import cv2
import matplotlib.pyplot as plt
from skimage.color import rgb2lab, lab2rgb

In [2]:
import numpy as np

def clahe(img, clip_limit=5.0, grid_size=(1,1)):
    # Get image dimensions
    height, width = img.shape[:2]
    # Divide the image into non-overlapping contextual regions
    grid_rows, grid_cols = grid_size
    grid_row_height = height // grid_rows
    grid_col_width = width // grid_cols
    # Create the output image
    clahe_img = np.zeros_like(img)
    # Perform CLAHE on each grid
    for row in range(grid_rows):
        for col in range(grid_cols):
            # Get the grid region
            grid_region = img[row*grid_row_height:(row+1)*grid_row_height, col*grid_col_width:(col+1)*grid_col_width]
            # Get the histogram of the grid region
            hist, _ = np.histogram(grid_region.ravel(), bins=256, range=(0, 255))
            # Clip the histogram
            clip_val = clip_limit * np.sum(hist)
            n_clip = np.minimum(hist, clip_val)
            # Calculate the average number of pixels to redistribute
            avg_pixels = (np.sum(hist) - np.sum(n_clip)) / 256
            # Redistribute the clipped pixels
            remain_pixels = avg_pixels
            for i in range(255, -1, -1):
                if n_clip[i] > 0:
                    redistribute = min(n_clip[i], remain_pixels)
                    hist[i] += redistribute
                    remain_pixels -= redistribute
            # Calculate the cumulative density function
            cdf = np.cumsum(hist)
            cdf = 255 * cdf / cdf[-1]
            # Map the original pixel values to the new values
            grid_region = np.interp(grid_region, np.arange(256), cdf)
            # Insert the grid region back into the output image
            clahe_img[row*grid_row_height:(row+1)*grid_row_height, col*grid_col_width:(col+1)*grid_col_width] = grid_region
    return clahe_img


def clahe2(image):
    hsv_image = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    hsv_image[:,:,2] = clahe(hsv_image[:,:,2],clip_limit = 5.0,grid_size = (1,1))
    
    image = cv2.cvtColor(hsv_image,cv2.COLOR_HSV2BGR)
    for i in range(3):
        image[:,:,i] = clahe(image[:,:,i],clip_limit = 5.0,grid_size = (1,1))
    return image

In [3]:
for i in range(1,19):
    img1 = cv2.imread('C://Users//bhari//Downloads//sip//dehazed//DCP_normal_'+str(i)+'.jpg')
    img2 = cv2.imread('C://Users//bhari//Downloads//sip//CAP//CAP_normal_'+str(i)+'.jpg')
    k=1
    output1 = clahe(img1)
    output2 = clahe2(img1)

    output3 = clahe(img2)
    output4 = clahe2(img2)
    
    cv2.imwrite('C://Users//bhari//Downloads//sip//CLAHE//DCP_Morphology//normalCLAHE_'+str(i)+'.jpg',output1)
    cv2.imwrite('C://Users//bhari//Downloads//sip//CLAHE//DCP_Morphology//normalmCLAHE_'+str(i)+'.jpg',output2)

    cv2.imwrite('C://Users//bhari//Downloads//sip//CLAHE//CAP_HLP//normalCLAHE_'+str(i)+'.jpg',output3)
    cv2.imwrite('C://Users//bhari//Downloads//sip//CLAHE//CAP_HLP//normalmCLAHE_'+str(i)+'.jpg',output4)